# Import from Excel

```{admonition} Authors
This notebook was prepared in October 2023 by [`@michaelweinold`](https://github.com/michaelweinold).
```
```{admonition} Download
You can download this notebook here: {nb-download}`import_excel.ipynb` \
You can download the required Excel sheet here: {nb-download}`my_own_database.xlsx`
```
```{admonition} Prerequisites
1. [A working installation of Brightway25.](https://docs.brightway.dev/en/latest/content/installation/index.html)
2. The Ecoinvent 3.9.1, cutoff database added to your Brightway project (here named `import_excel`).
```

In [ ]:
# Brightway packages
import bw2io as bi
import bw2data as bd
import bw2calc as bc

In [39]:
bd.projects.set_current("import_excel")

Here, we assume you have added the Ecoinvent 3.9.1 database to your Brightway project.

In [42]:
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.9.1-biosphere
	ecoinvent-3.9.1-cutoff

We can now import an Excel sheet into Brightway. This is useful if you have data in Excel format that you want to use in your LCA calculations.

Creating such an Excel sheet is simple: Follow the basic example in `my_own_database.xlsx` and add your own data.

As you can see, you must separate **Activity** blocks by a single empty row. The name of the individual sheets does not matter. Brightway attempts to import all sheets in the Excel file. In the example case, there is only a single sheet, named `data`. You may stop Brightway from importing specific sheets by writing `skip` in the first cell of the sheet.


In [53]:
my_db = bi.ExcelImporter("my_own_database.xlsx")

Extracted 1 worksheets in 0.01 seconds


In [54]:
my_db.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 5.50 seconds


In [55]:
my_db.match_database(
    "ecoinvent-3.9.1-cutoff", fields=["name", "unit", "location", "reference product"]
)

Applying strategy: link_iterable_by_fields


In [56]:
my_db.statistics()

2 datasets
	4 exchanges
	Links to the following databases:
		my_own_database (3 exchanges)
		ecoinvent-3.9.1-cutoff (1 exchanges)
	0 unlinked exchanges (0 types)
		


(2, 4, 0, 0)

In [57]:
my_db.write_database()

100%|██████████| 2/2 [00:00<00:00, 5461.33it/s]


Vacuuming database 
Created database: my_own_database


In [58]:
bd.databases

Databases dictionary with 3 object(s):
	ecoinvent-3.9.1-biosphere
	ecoinvent-3.9.1-cutoff
	my_own_database

We can now browse the activities in the database...

In [59]:
[activitiy for activitiy in bd.Database("my_own_database")]

['magic ingredient production' (kilogram, CO, None),
 'magic banana production' (kilogram, CO, None)]

...and conduct a basic life-cycle assessment of our new activity:

In [61]:
act

'magic banana production' (kilogram, CO, None)

In [71]:
banana_activity = [
    activitiy for activitiy in bd.Database("my_own_database")
    if 'magic banana' in activitiy['name']
]

In [73]:
ipcc_gwp_method = [
    method for method in bd.methods
    if 'IPCC 2021' == method[1]
    and 'climate change' == method[2]
    and '(GWP100)' in method[3]
][0]

In [74]:
my_functional_unit, data_objs, _ = bd.prepare_lca_inputs(
    {act: 42},
    method=ipcc_gwp_method)
my_lca = bc.LCA(demand=my_functional_unit, data_objs=data_objs)
my_lca.lci()
my_lca.lcia()

In [75]:
my_lca.score

0.016015050756031914